<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

I have chosen to create a class which will include the necessary functions to geocode and process the collected data.

First, let's import the necessary libraries.

In [1]:
import json
import xml
import requests
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import geocoder

<h2>PART 1</h2>
In this part I am creating a class that will do the actual work.

Functions are the following:
<ul>
 	<li><strong>scrape</strong> for web scraping data from Wikipedia</li>
 	<li><strong>savetoCSV </strong>to save the data to a file</li>
 	<li><strong>geocodetheCoords</strong> to convert the postal data to latitude and longitude.</li>
    <li><strong>ifGeocoderFails</strong> to import latitudes and longitudes from a provided file. This is needed in case Geocoder fails (in my case, it sometimes failed to deliver). Then I use pd.merge to relate the coordinates with the scraped dataset.</li>
</ul>

In [2]:
class Scraper:
    postcode, borough, neighborhood = [], [], []
    
    def __init(self, df):
        self.df = df
        
    def scrape(self, url):
        soup = BeautifulSoup(url,'xml')
        table = soup.find('table')

        self.postcode, self.borough, self.neighborhood = [], [], []

        for tr_cell in table.find_all('tr'):
            
            count = 1
            PostcodeInd, BoroughInd, NeighborhoodInd = -1, -1, -1
            
            for td_cell in tr_cell.find_all('td'):
                if count == 1: 
                    PostcodeInd= td_cell.text
                if count == 2: 
                    BoroughInd= td_cell.text
                    tag_a_Borough = td_cell.find('a')
                    
                if count == 3: 
                    NeighborhoodInd = str(td_cell.text).strip()
                    tag_a_Neighborhood = td_cell.find('a')
                    
                count +=1
                
            if (PostcodeInd== 'Not assigned' or BoroughInd== 'Not assigned' or NeighborhoodInd == 'Not assigned'): 
                continue
            try:
                if ((tag_a_Borough is None) or (tag_a_Neighborhood is None)):
                    continue
            except:
                pass
            if(PostcodeInd== -1 or BoroughInd == -1 or NeighborhoodInd == -1):
                continue
                
            self.postcode.append(PostcodeInd)
            self.borough.append(BoroughInd)
            self.neighborhood.append(NeighborhoodInd)
            
        uPostal = set(self.postcode)
        print('Number of unique postal codes:', len(uPostal))
        uBorough, uNeighborhood, uPostcode = [], [], []
        
        for pcElement in uPostal:
            p = ''; b = ''; n = ''; 
            for postcode_idx, postcode_element in enumerate(self.postcode):
                if pcElement  == postcode_element:
                    p = postcode_element;
                    b = self.borough[postcode_idx]
                    if n == '': 
                        n = self.neighborhood[postcode_idx]
                    else:
                        n = n + ', ' + self.neighborhood[postcode_idx]
            uPostcode.append(p)
            uBorough.append(b)
            uNeighborhood.append(n)
            
        temp = {'Postcode': uPostcode, 'Borough' :uBorough, 'Neighborhood' :uNeighborhood}
        self.df = pd.DataFrame.from_dict(temp)
        
        return self.df, uPostcode
    
    def savetoCSV(self):
        self.df.to_csv('torontoData.csv')

    def geocodetheCoords(self):
        
        latitude = []
        longitude = []
        for elem in t[1]:
            lat_lng_coords = None

            while (lat_lng_coords is None):
                g = geocoder.google('{}, Toronto, Ontario'.format(elem))
                lat_lng_coords = g.latlng
                # print(lat_lng_coords)
        
            latitude.append(lat_lng_coords[0])
            longitude.append(lat_lng_coords[1])
            print(elem, 'is saved.')
        return latitude, longitude
    
    def ifGeocoderFails(self):
        importFromFile = pd.read_csv("https://cocl.us/Geospatial_data")
        return importFromFile

<h2>PART 2</h2>
In this part I execute the above-prescribed actions.

In [3]:
Toronto = Scraper()
t = Toronto.scrape(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text)


Toronto.savetoCSV()
print(t[0].head())

Number of unique postal codes: 84
            Borough                               Neighborhood Postcode
0  Downtown Toronto  Chinatown, Grange Park, Kensington Market      M5T
1  Downtown Toronto                             Commerce Court      M5L
2       Scarborough                  Agincourt North, Milliken      M1V
3         East York                                    Leaside      M4G
4         East York                           Woodbine Heights      M4C


In [4]:
#coords = Toronto.geocodetheCoords()
#t[0]['Latitude'] = coords[0]
#t[0]['Longitude'] = coords[1]

if geocoder fails, let's import the coordinates from the file.

In [5]:
coords = Toronto.ifGeocoderFails()
print(coords.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [6]:
#Let's sort the data 
coords.sort_values(by=['Postal Code'])
coords.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
t[0].sort_values(by=['Postcode'])

#t[0]['Latitude'] = coords['Latitude']
#t[0]['Longitude'] = coords['Longitude']

expected_result = pd.merge(t[0], coords, on = 'Postcode', how = 'left')

In [9]:
from IPython.display import display, HTML
display(expected_result)

,Borough,Neighborhood,Postcode,Latitude,Longitude
0,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",M5T,43.653206,-79.400049
1,Downtown Toronto,Commerce Court,M5L,43.648198,-79.379817
2,Scarborough,"Agincourt North, Milliken",M1V,43.815252,-79.284577
3,East York,Leaside,M4G,43.709060,-79.363452
4,East York,Woodbine Heights,M4C,43.695344,-79.318389
5,Scarborough,"Maryvale, Wexford",M1R,43.750072,-79.295849
6,Downtown Toronto,University of Toronto,M5S,43.662696,-79.400049
7,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
8,North York,Downsview West,M3L,43.739015,-79.506944
9,Central Toronto,Moore Park,M4T,43.689574,-79.383160
